In [1]:
import pandas as pd
import numpy as np
import openpyxl as ox
import re
import editdistance
from datetime import datetime

In [635]:
# надо ли переделать на создание без извлечения из иксель?
df = pd.DataFrame(columns=['Менеджер', 'Вид доставки', 'Дата', 'Товар', 'Примечание к Товару', 
                           'Кол-во', 'Ед.изм.', 'Машина/Водитель','Продавец', 'Откуда', 
                           'Покупатель', 'Грузополучатель', 'Адрес грузополучателя (юрид)', 
                           'Адрес пункта разгрузки', 'Контакт гп','Время приемки',
                          ])


In [381]:
df

,Менеджер,Вид доставки,Дата,Товар,Примечание к Товару,Кол-во,Ед.изм.,Машина/Водитель,Продавец,Откуда,Покупатель,Грузополучатель,Адрес грузополучателя (юрид),Адрес пункта разгрузки,Контакт гп,Примечание Иное


In [4]:
managers = pd.read_excel("dictionary.xlsx", sheet_name="managers")
products = pd.read_excel("dictionary.xlsx", sheet_name="products")
units = pd.read_excel("dictionary.xlsx", sheet_name="units")
unload_addresses = pd.read_excel("dictionary.xlsx", sheet_name="unload_addresses")

In [125]:
# managers = pd.DataFrame({'login': ['ИГО', 'Юра Менеджер', 'Алексей Мельхер'],
#                         'manager': ['Игорь Хабаров', 'Юрий', 'Алексей Мельхер']})

In [539]:
# units = pd.DataFrame({'data': ['т', 'т.', 'тн', 'тонн', 'кубов'],
#                         'unit': ['т', 'т', 'т', 'т', 'куб']})

In [126]:
# products = pd.Series(data=['ЦЕМ I 42.5Н БЦК','Щебень гранитный 5-20 (ЛСР)','ЦЕМ I 42.5Н Костюковичи',
#                                'ЦЕМ I 42.5Н БЦЗ Костюковичи', 'ЦЕМ I 42.5Н', 'ПЕСКОГРУНТ'], name='product')

In [536]:
# unload_addresses = pd.DataFrame({'pattern1': ['Бетонная индустрия', 'Бетон Строй', 'МСУ-1', 'БетонПром', 
#                                            'Сибирский элемент Рента', 'КАНТАН', 'М Бетон', '"ТД"Цемент'], 
#                         'pattern2': ['Одинцово', 'Солнечногорск', 'Бескудниковский', 'Октябрьский', 'Обухово', 
#                                      'Соколовское', 'Очаковское','Голицыно'],
#                         'address': ['Одинцово. Кобяковская. Краснознаменск.', 
#                                     'город Солнечногорск, ул. Промышленная 1б ', 
#                                     'Москва, Бескудниковский бульвар, 52к1',
#                                     'Октябрьский, улица Старый двор, промзона',
#                                     'Калужская обл., Дзержинский р-н, д. Обухово 1Б',
#                                     'Московская обл., территориальное упр-е Соколовское, г.о. Солнечногорск',
#                                     'Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината',
#                                     'Голицыно, Можайское ш., 81, РБУ'
#                                    ],
#                               })

In [540]:
with pd.ExcelWriter("dictionary.xlsx") as writer:
    managers.to_excel(writer, sheet_name="managers", index=False)
    products.to_excel(writer, sheet_name="products", index=False)
    units.to_excel(writer, sheet_name="units", index=False)
    unload_addresses.to_excel(writer, sheet_name="unload_addresses", index=False)

In [554]:
products['product'].str.replace(' ', '').replace('.', '')

0                ЦЕМI42.5НБЦК
1    Щебеньгранитный5-20(ЛСР)
2        ЦЕМI42.5НКостюковичи
3     ЦЕМI42.5НБЦЗКостюковичи
4                   ЦЕМI42.5Н
5                  ПЕСКОГРУНТ
6             ЦемI42.5НКричев
7           ЦЕМI42.5НБеларусь
Name: product, dtype: object

## Text

In [719]:
appl = """
ИГО:
Заявка на доставку
1. Дата отгрузки на 26.04
2. Марка цемента ЦЕМ I 42,5н
Сертификаты!
3. Количество 35 тонн 
4. Продажа от ООО "Спарта"
5. С Солнечногорска  
6. Покупатель ООО "Бетон Строй»
7. Грузополучатель ООО "Бетон Строй»
8. Адрес город Солнечногорск, ул. Промышленная 1б 
+79232208000
+74959242828
Дебет оплата 250
У162ХК750 MAN
Х240ВА797 КАМАЗ
А215УХ750 МAN
8. Адрес грузополучателя (юрид): 249850, Калужская обл., Дзержинский р-н, д. Обуховской, д 1Б. 
8. Время приёмки: с 9.00 до 20.00
"""

In [661]:
# Используем регулярное выражение для поиска даты в формате dd.mm.yyyy
date_pattern = r'\b\d{2}\.\d{2}(?:\.\d{2}(?:\d{2})?)?\b'

# Находим все совпадения с помощью метода findall
dates = re.findall(date_pattern, appl)

## Check

In [662]:
dates

['26.04']

In [663]:
# функция для форматирования даты в установленный формат
def convert_to_full_year(date_str):
    current_year = datetime.now().year
    parts = date_str.split('.')
    
    # Пытаемся интерпретировать дату
    try:
        # Если в строке только две части, добавляем текущий год
        if len(parts) == 2:
            date_str += f".{current_year}"
        
        # Пробуем преобразовать строку в дату
        datetime_obj = datetime.strptime(date_str, '%d.%m.%Y')
    except ValueError:
        # Если возникает ошибка, попробуем считать год в формате yy
        try:
            datetime_obj = datetime.strptime(date_str, '%d.%m.%y')
        except ValueError:
            # Если и это не сработает, возможно, формат даты неизвестен
            raise ValueError("Неподдерживаемый формат даты")
    
    # Возвращаем дату в формате dd.mm.yyyy
    return datetime_obj.strftime('%d.%m.%Y')

In [664]:
# конвертируем ПОКА ЧТО для одной даты
convert_date = convert_to_full_year(dates[0])

## Check

In [665]:
convert_date

'26.04.2024'

In [666]:
def find_manager(text, managers_df):
    # Проходим по каждому логину менеджера
    for index, row in managers_df.iterrows():
        login = row['login']
        manager = row['manager']
        
        # Проверяем совпадение по колонке 'login'
        if login.lower().replace(' ', '') in text.lower().replace(' ', ''):
            return manager
    return np.NAN



In [667]:
manager_found = find_manager(appl, managers)

In [668]:
manager_found

'Игорь Хабаров'

In [669]:
def delivery_type(text):
    if "самовывоз" in text.lower():
        return "самовывоз"
    return "доставка"

In [670]:
delivery_found = delivery_type(appl)

In [671]:
delivery_found

'доставка'

In [209]:
# def find_product(text, product_df):
#     """
#     Функция для поиска товара по тексту и датафрейму с видами товара.

#     Аргументы:
#     text (str): Текст для поиска товара.
#     product_df (pandas.DataFrame): Датафрейм с колонками 'login' и 'manager',
#                                      содержащий информацию о менеджерах.

#     Возвращает:
#     str: Имя менеджера, если совпадение найдено, иначе сообщение о том, что совпадений не найдено.
#     """
#     # Проходим по каждому логину менеджера
#     for index, row in product_df.iterrows():
#         product = row['product']
#         # Проверяем совпадение по колонке 'login'
#         if product.lower().replace(' ', '') in text.lower().replace(' ', ''):
#             return product
#     # Если совпадений не найдено
#     return np.NAN

In [210]:
# product_found = find_product(appl, products)

In [211]:
# product_found

In [678]:
def find_product(text):
    # Находим значение после слова "Марка:" без учета регистра
    marca_match = re.search(r'(?i)Марка(:)?\s*(цемента)?\s*(.*?)\n', text)
    marca_value = marca_match.group(3) if marca_match else np.NAN

    return marca_value
    
def find_product_notice(text):    
    # Находим значение между строкой с "Марка:" и следующим номером пункта
    # Если нет следующего номера пункта, возвращаем np.NAN
  
    intermediate_match = re.search(r'(?i)Марка.*?\n(.*?)\n\d', text, re.DOTALL)
    intermediate_value = intermediate_match.group(1) if intermediate_match else np.NAN
    
    if re.match(r'^[^0-9.].*', intermediate_value):
        return intermediate_value
    else:
        np.NAN

In [679]:
product_found = find_product(appl)

In [680]:
product_notice_found = find_product_notice(appl)

## Check

In [681]:
product_found

'ЦЕМ I 42,5н'

In [682]:
product_notice_found

'Сертификаты!'

In [683]:
def find_quantity(text):
    # Находим значение после слова "Количество/кол-во:" без учета регистра    
    quantity_match = re.search(r'(?i)кол(\s*-\s*|ичест)?во\s*(:)?\s*(\d+)', text)
    quantity = int(quantity_match.group(3)) if quantity_match else np.NAN
    
    return quantity

In [684]:
quantity_found = find_quantity(appl)

In [685]:
quantity_found

35

In [686]:
def find_unit_note(text):
    # Находим значение после слова "Количество/кол-во:" без учета регистра    
    unit_match = re.search(r'(?i)кол(-|ичест)?во\s*(:)?\s*(\d+)\s*(\D+)\s*', text)
    unit = unit_match.group(4).strip() if unit_match else np.NAN
    
    return unit

In [687]:
unit_note_found = find_unit_note(appl)

In [688]:
unit_found = units.loc[units[units['data']==unit_note_found].index[0], 'unit']

In [693]:
def find_car(text):
    
    car_match = re.findall(r'(?i)[А-Я]{1}\d{3}[А-Я]{2}\d{3}\s*\w*', text)
    cars = car_match if car_match else np.NAN
    
    return cars

In [694]:
cars_found = ', '.join(find_car(appl)) if find_car(appl) is not np.NAN else np.NAN

In [695]:
cars_found

'У162ХК750 MAN, Х240ВА797 КАМАЗ, А215УХ750 МAN'

In [696]:
def find_organization(text):
#     Продажа от: ООО "Спарта"
    organization_match = re.search(r'(?i)(продажа\s+от:|(продажа)?\s*от\s+(клиента)?\s*(:)?)\s*(.+?)\n', text)
    organization = organization_match.group(5) if organization_match else np.NAN
    
    return organization

In [697]:
organization_found = find_organization(appl)

In [698]:
organization_found

'ООО "Спарта"'

In [699]:
def find_transshipment(text):
    
    transshipment_match = \
    re.search(r'(?i)\d+\.\s*(С\s+(перевалки)?\s*|Завод\s*(отгрузки)?\s*(:)?|Перевалка\s*(:)?)\s*(.+?)\n', 
              text)
    transshipment = transshipment_match.group(6) if transshipment_match else np.NAN
    
    return transshipment

In [700]:
transshipment_found = find_transshipment(appl)

In [701]:
transshipment_found 

'Солнечногорска  '

## Доделать, спросить у Лёхи

Нужно ли создавать словарь для именительного падежа названия перевалки?

In [702]:
def find_purchaser(text):
    
    purchaser_match = \
    re.search(r'(?i)\d+\.\s*(Покупатель\s*(груза)?\s*(:)?)\s*(.+?)\n', 
              text)
    purchaser = purchaser_match.group(4) if purchaser_match else np.NAN
    
    return purchaser

In [703]:
purchaser_found = find_purchaser(appl)

In [704]:
purchaser_found

'ООО "Бетон Строй»'

  ## Доделать, спросить у Лёхи

4. ООО «ПД Технологии» объект *Пушкино* ,
4. ООО «ПД Технологии» объект *Пушкино* 

In [705]:
def find_consignee(text):
    
    consignee_match = \
    consignee_match = re.search(r'(?i)\d+\.\s*(?:Грузополучатель\s*(?::)?|Грузополучатель\s*\(при\s*оформлении\s*ттн\)\s*(?::)?)\s*(.+?)\n', 
              text)
    consignee = consignee_match.group(1).split(':')[-1].strip() if consignee_match else np.NAN
    
    return consignee

In [706]:
consignee_found = find_consignee(appl)

In [707]:
consignee_found

'ООО "Бетон Строй»'

## Лёхе

С этим надо что-то делать, так нихуя не понятно  
5. Бетас 

In [709]:
def find_consignee_leg_addr(text):
    
    find_consignee_leg_addr_match = \
    re.search(r'(?i)\d+\.\s*(Юр\s*\.)?\s*(Адрес\s*грузополучателя)\s*(\(юрид\))?\s*(:)?\s*(.+?)\n',
              text)
    find_consignee_leg_addr = find_consignee_leg_addr_match.group(5) if find_consignee_leg_addr_match else np.NAN
    
    return find_consignee_leg_addr

In [710]:
consignee_leg_addr_found = find_consignee_leg_addr(appl)

In [711]:
consignee_leg_addr_found

'249850, Калужская обл., Дзержинский р-н, д. Обуховской, д 1Б.    '

## Лёхе
Может ли быть подъёб со стороны менеджера, который еще вариант написания придумает?  
8. Адрес грузополучателя (юрид):  
8. Юр.Адрес грузополучателя:

In [712]:
def find_phones(text):
    phone_numbers = re.findall(r'\+?\d{1,3}[\s-]?\(?\d{3}\)?[\s-]?\d{2,3}[\s-]?\d{2}[\s-]?\d{2}(?=\s|$)', text)
    phone_numbers = phone_numbers if phone_numbers else np.NAN
    
    return phone_numbers

In [713]:
phones_found = find_phones(appl)

In [714]:
phones_found = ', '.join(find_phones(appl)) if find_phones(appl) is not np.NAN else np.NAN

In [715]:
phones_found

'+79232208000, +74959242828'

In [716]:
def find_unl_addr(text, unload_addresses_df):
    # Проходим по каждому логину менеджера
    for index, row in unload_addresses_df.iterrows():
        pattern1 = row['pattern1']
        pattern2 = row['pattern2']
        address = row['address']
        
        # Проверяем совпадение по колонке 'login'
        if pattern1.lower().replace(' ', '') and pattern2.lower().replace(' ', '') \
        in text.lower().replace(' ', ''):
            return address
    return np.NAN

In [717]:
unl_addr_found = find_unl_addr(appl, unload_addresses)

In [718]:
unl_addr_found

'город Солнечногорск, ул. Промышленная 1б '

## Лёхе

Забить все адреса в словарь

In [720]:
def find_time(text):
    if 'приёмк' or 'приемк' in text.lower():
        time_match = re.search(r'(?i)(время)?\s*при(ё|е)мк(и|а)(?::)?\s*(.*)', text)
        time = time_match.group(4) if time_match else np.NAN
    
    return time

In [721]:
accept_time_found = find_time(appl)

In [722]:
accept_time_found

'с 9.00 до 20.00'

In [632]:
text = """
Юра Менеджер:
Заявка на самовывоз
1. Дата отгрузки 28.04.2023 
2. Марка ЦЕМ I 42.5н Беларусь   
3. Количество 40 тонн 
9. Приёмка иди в пизду
10. jdjksdfj
"""

In [ ]:
Дебет оплата 250
8. Время приёмки: к 9 

In [723]:
# Создаем новый DataFrame с одной строкой, содержащей только дату
new_row_df = pd.DataFrame({'Менеджер': [manager_found], 
                           'Вид доставки': [delivery_found], 
                           'Дата': [convert_date], 
                           'Товар': [product_found], 
                           'Примечание к Товару': [product_notice_found], 
                           'Кол-во': [quantity_found], 
                           'Ед.изм.': [unit_found], 
                           'Машина/Водитель': [cars_found], 
                           'Продавец': [organization_found], 
                           'Откуда': [transshipment_found], 
                           'Покупатель': [purchaser_found], 
                           'Грузополучатель': [consignee_found],
                           'Адрес грузополучателя (юрид)': [consignee_leg_addr_found], 
                           'Адрес пункта разгрузки': [unl_addr_found], 
                           'Контакт гп': [phones_found],
                           'Время приемки': [accept_time_found],
#                            'Примечание Иное',
                          })

# new_row_df['Машина/Водитель'] = new_row_df['Машина/Водитель'].apply(lambda x: ', '.join(x)) 
# Объединяем исходный DataFrame с новым DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)

In [724]:
df

,Менеджер,Вид доставки,Дата,Товар,Примечание к Товару,Кол-во,Ед.изм.,Машина/Водитель,Продавец,Откуда,Покупатель,Грузополучатель,Адрес грузополучателя (юрид),Адрес пункта разгрузки,Контакт гп,Время приемки
0,Юрий,самовывоз,27.04.2024,ПЕСКОГРУНТ,Обязательно актуальный паспорт!,200,куб,"О327ВН790 МАN, О039ОУ790 МАN, Х194ВА797 КАМАЗ,...",ООО Спарта,"КПП, Солнечногорск, Люберцы","ООО ""КАНТАН""",ООО БетонПром,"249850, Калужская обл., Дзержинский р-н, д. Об...","Голицыно, Можайское ш., 81, РБУ","+79777728931, 8-925-758-01-54, 8-925-758-01-54...",с 9.00 до 20.00
1,Игорь Хабаров,доставка,26.04.2024,"ЦЕМ I 42,5н",Сертификаты!,35,т,"У162ХК750 MAN, Х240ВА797 КАМАЗ, А215УХ750 МAN","ООО ""Спарта""",Солнечногорска,"ООО ""Бетон Строй»","ООО ""Бетон Строй»","249850, Калужская обл., Дзержинский р-н, д. Об...","город Солнечногорск, ул. Промышленная 1б","+79232208000, +74959242828",с 9.00 до 20.00


In [75]:
df.drop(df.index, inplace=True)

In [76]:
df.columns

Index(['Менеджер', 'Вид доставки', 'Дата', 'Товар', 'Примечание к Товару',
       'Кол-во', 'Ед.изм.', 'Машина/Водитель', 'Продавец', 'Откуда',
       'Покупатель', 'Грузополучатель ', 'Адрес грузополучателя (юрид)',
       'Адрес пункта разгрузки', 'Контакт гп', 'Примечание Иное'],
      dtype='object')

In [ ]:
patterns = [
    # ПАО ГТС => ГТС, ПАО
    r'^(?i)(П?АО|ООО|ГУП)(?:\s+|\s*")([^"]+)"?',
    
    # ПАО Р-БАНК//ИЗМАЙЛОВ ВАЛЕРИЙ МИХАЙЛОВИЧ//РОССИЯ,МОСКВА Г Р/С в ПАО Р-БАНК г Москва => Измайлов В.
    r'//([\w-]+)\s+(\w)\w*\s+(?:\w)\w*//',
    
    # Индивидуальный ПРЕДПРИНИМАТЕЛЬ Логинов Андрей Николаевич => Логинов А., ИП
    r'^(?i)(и)(?:ндивидуальный)?\s*(п)(?:редприниматель)?\s*(\w+)\s+(\w)\w*\s*(\w)?\w*',
    
    # Никишкина Лидия Алексеевна (ИП) => Нокошина Л., ИП
    r'^([\w-]+)\s+(\w)\w*\s*(\w)?\w*\s*\((ИП)\)$',
    
    # ВОРОНОВ ИЛЬЯ АНДРЕЕВИЧ => Воронов И.
    r'([\w-]+)\s+(\w)\w*\s+(\w)\w*$',
    
    # ООО "ЧОП "ОХРАННЫЕ СИСТЕМЫ" => ОХРАННЫЕ СИСТЕМЫ, ЧОП, ООО
    r'^([\w-]{2,3})\s+"?(.*)\s+"(.*)"',
    
    #  contains "ФНС": 
    r'\w*\s*(?i)(?:и?)\s*(ф)(?:едеральной)?\s*(н)(?:алоговой)?\s*(с)(?:лужбы)?\s*(?:россии)?\s*\w*\s*', 
    
    # ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЛАГ1" => ЛАГ1, ООО
    r'(\w)\w{3,}|\"([^\"]+)\"',
    
    # Межрегиональное операционное УФК (КАЗНАЧЕЙСТВО РОССИИ) => Межрегиональное операционное УФК
    r'^.+?(?=\s*\()',
]       

def extract_text(row):
    """
    Get string with raw title, extract necessary text by patterns and returns formatted values
    """
    matches = []

    for pattern in patterns:
        match = re.findall(pattern, row)
        matches.append(match)
    
    return '{}, {}, {}'.format(matches[5][0][2], matches[5][0][1], matches[5][0][0]) if matches[5] else \
        '{} {}.'.format(matches[1][0][0].capitalize(), matches[1][0][1]) if matches[1] else \
        '{}, {}'.format(matches[0][0][1], matches[0][0][0]) if matches[0] else \
        '{} {}.{}., {}{}'.format(matches[2][0][2].capitalize(), matches[2][0][3], matches[2][0][4], matches[2][0][0], matches[2][0][1]) if matches[2] else \
        '{} {}.{}., {}'.format(matches[3][0][0].capitalize(), matches[3][0][1], matches[3][0][2], matches[3][0][3]) if matches[3] else \
        '{}{}{}'.format(matches[6][0][0].upper(),matches[6][0][1].upper(),matches[6][0][2]).upper() if matches[6] else \
        '{} {}.{}.'.format(matches[4][0][0].capitalize(), matches[4][0][1], matches[4][0][2]) if matches[4] else \
        '{}, {}'.format([mat[1] for mat in matches[7] if mat[1]][0],
                        ''.join(map(str, [mat[0] for mat in matches[7] if
                                          mat[0]]))) if '"' in row else \
        '{}'.format(matches[8][0]) if matches[8] else \
        row

# creating column "Agent"
df['Agent'] = np.vectorize(extract_text)(df['Full_Name'])
df['Name'] = df['Agent'].map(lambda x: x.lower())

In [95]:
def fix_typo(original, target):
    distance = editdistance.eval(original, target)
    if distance <= 3:  # Пороговое значение для определения опечатки
        return target
    else:
        return original

original = "Щебин ранитный"
target = "Щебень гранитный"
fixed_string = fix_typo(original, target)
print("Fixed string:", fixed_string)

Fixed string: Щебень гранитный


In [139]:
import re
import numpy as np

text = """
1. Дата отгрузки: 28.04.24 
2. Марка Цем I 42.5Н Кричев
3. Количество: 40т.
"""

# Находим значение после слова "Марка:" без учета регистра
marca_match = re.search(r'Марка\s*(цемента)?\s*(.*?)\n', text, re.IGNORECASE)
marca_value = marca_match.group(2)

# Находим значение между строкой с "Марка:" и следующим номером пункта
# Если нет следующего номера пункта, возвращаем np.NAN
intermediate_match = re.search(r'Марка.*?\n(.*?)\n\d', text, re.IGNORECASE | re.DOTALL)
intermediate_value = intermediate_match.group(1) if intermediate_match else np.NAN

print("Значение после слова 'Марка:':", marca_value)
print("Значение между строкой с 'Марка:' и следующим номером:", intermediate_value)


Значение после слова 'Марка:': Цем I 42.5Н Кричев
Значение между строкой с 'Марка:' и следующим номером: nan


In [64]:
pattern = r'\d+\..*'
text = "5. Надо полис"

match = re.match( r'\d+\..*', text)

In [65]:
match[0]

'5. Надо полис'

In [83]:
type(re.match(r'^[^0-9.].*', "5. Надо полис"))

NoneType

In [67]:
import re

text="""
2. Марка: ЦЕМ I 42,5н БЦЗ Костюковичи 
Машина с пломбами и актуальными паспортами 
3. Количество: 40 тю.
4. Продажа от: ООО "Спарта"
"""

# Ищем строку, содержащую "Количество:"
match = re.search(r'(?i)кол(-|ичест)?во\s*(:)?\s*(\d+)\s*(\D+)\s*', text)
# Если найдено соответствие, выводим символы после числа в этой строке
if match:
    print(match.group(4))


тю.



In [83]:
import re

def find_unit(text):
    # Находим значение после слова "Количество/кол-во:" без учета регистра    
    unit_match = re.search(r'(?i)кол(-|ичест)?во\s*(:)?\s*\d+\s*(\D+)\s*', text)
    unit = unit_match.group(3) if unit_match else None
    
    return unit

appl = """
Юра Менеджер:
1. Дата отгрузки:
26.04                    
2. От ООО Спарта
3. Марка: Щебень гранитный 5-20(ЛСР)
Обязательно актуальный паспорт!
4. Покупатель ООО НВЛ ГРУП
5. Бетас 
6.  Количество: 120т
7. Машина: 
О327ВН790 МАN
"""

print(find_unit(appl))


т



In [150]:
import re

text="""
4. Покупатель ООО НВЛ ГРУП
5. Бетас 
6.  Количество 120т
7. Машина: 
О327ВН790 МАN
О039ОУ790 МАN,
х194ВА797 КАМАЗ
В683СН790 МАN
К702ОС797 Куряев
"""

car_numbers = re.findall(r'(?i)[А-Я]{1}\d{3}[А-Я]{2}\d{3}\s*\w*', text)

print(car_numbers)


['О327ВН790 МАN', 'О039ОУ790 МАN', 'х194ВА797 КАМАЗ', 'В683СН790 МАN', 'К702ОС797 Куряев']


In [170]:
import pandas as pd

# Создание DataFrame с вашим списком в одной из ячеек
df = pd.DataFrame({'col_name': [['abc', 'def', 'ghi']]})

# Применение метода str.join() к списку в ячейке и замена значения ячейки
df['col_name'] = df['col_name'].apply(lambda x: ', '.join(x))

# Вывод измененного DataFrame
print(df)


        col_name
0  abc, def, ghi


In [171]:
df = pd.DataFrame({'col_name': [['abc', 'def', 'ghi']]})

In [173]:
df

,col_name
0,"[abc, def, ghi]"


In [121]:
import re

text = "5. Покупатель ООО НВЛ ГРУП\n5"

result = re.search(r'\d+\.\s(.+?)\n', text, re.DOTALL)
if result:
    extracted_text = result.group(1)
    print(extracted_text)


Покупатель ООО НВЛ ГРУП


In [485]:
import re

text = """
Контакт диспетчера: 8 (903) 007-00-86\n+79232208000\n+74959242828\n9. Контакт диспетчера: +7 968 558-83-13 \nКонтактный телефон 8 (965) 709-90-77 Игорь.\n9. Контакт диспетчера: +79777728931\n10. Контакты для связи 8-925-758-01-54\n8. Голицыно\n+7 910 404-06-14\n10.
"""

# Регулярное выражение для поиска номеров телефонов
phone_regex = re.compile(r'(\+?[\d\s()-]+)')

# Извлечение номеров телефонов из текста
phone_numbers = phone_regex.findall(text)

# Печать найденных номеров телефонов
for number in phone_numbers:
    print(number.strip())




8 (903) 007-00-86
+79232208000
+74959242828
9



+7 968 558-83-13

8 (965) 709-90-77
9



+79777728931
10



8-925-758-01-54
8


+7 910 404-06-14
10



In [494]:
import re

# Исходная строка
phone_number_with_extra = '+79777728931\n10'

# Шаблон для поиска номера телефона
pattern = r'\+\d+'

# Поиск совпадений с помощью регулярного выражения
matches = re.findall(pattern, phone_number_with_extra)

# Если найдено совпадение, берем первое (и, возможно, единственное)
if matches:
    phone_number = matches[0]
    print(phone_number)
else:
    print("Номер телефона не найден")


+79777728931


In [558]:
import re

def find_time_availability(text):
    # Паттерн для поиска времени приёмки
    time_pattern = r'Время приёмки:\s*([\d:.]+)'
    # Паттерн для поиска круглосуточной приёмки
    round_the_clock_pattern = r'Приёмка круглосуточно'

    # Поиск времени приёмки
    time_match = re.search(time_pattern, text)
    if time_match:
        time_availability = time_match.group(1)
        return f"Время приёмки: {time_availability}"
    
    # Поиск круглосуточной приёмки
    round_the_clock_match = re.search(round_the_clock_pattern, text)
    if round_the_clock_match:
        return "Приёмка круглосуточно"

    return "Информация не найдена"

# Пример текста
text = """
Юра Менеджер:
Заявка на самовывоз
1. Дата отгрузки 28.04.2023 
8. Время приёмки: с 9.00 до 20.00
2. Марка ЦЕМ I 42.5н Беларусь   
3. Количество 40 тонн 
9. Приёмка круглосуточно
"""

# Поиск и вывод информации
print(find_time_availability(text))
8. Время приёмки: с 9.00 до 20.00

Приёмка круглосуточно


In [609]:
pattern = r'(?i)(время)?\s*приёмк(и|а)(?::)?\s*(.*)'
# text = '\nЮра Менеджер:\nЗаявка на самовывоз\n1. Дата отгрузки 28.04.2023 \n8. Время приёмки: с 9.00 до 20.00\n2. Марка ЦЕМ I 42.5н Беларусь   \n3. Количество 40 тонн \n9. Приёмка круглосуточно\n'
text = """
Контактный телефон 8 (965) 709-90-77 Игорь.
Приёмка круглосуточно
Заявка на самовывоз
"""
match = re.search(pattern, text)

if match:
    print("Время приёмки", match.group(3).strip())
else:
    print("Время приёмки не найдено.")

Время приёмки круглосуточно


In [593]:
text = """
Контактный телефон 8 (965) 709-90-77 Игорь.
Время приёмки круглосуточно
Заявка на самовывоз
"""

In [594]:
text

'\nКонтактный телефон 8 (965) 709-90-77 Игорь.\nВремя приёмки круглосуточно\nЗаявка на самовывоз\n'